##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Retreine um modelo de reconhecimento de fala com o TensorFlow Lite Model Maker


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/speech_recognition"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Neste notebook do Colab, você aprenderá a usar o [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) (criador de modelos do TF Lite) para treinar um modelo de reconhecimento de fala que consegue classificar palavras ou frases curtas faladas usando amostras de som de um segundo. A biblioteca Model Maker usa aprendizado por transferência para retreinar um modelo do TensorFlow com um novo dataset, reduzindo a quantidade de dados de amostra e o tempo necessários para o treinamento.

Por padrão, este notebook retreina o modelo (BrowserFft, do [TFJS Speech Command Recognizer](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer)) usando um subconjunto de palavras do [dataset de comandos de voz](https://www.tensorflow.org/datasets/catalog/speech_commands) (como "up" – cima, "down" – baixo, "left" – esquerda e "right" – direita). Em seguida, ele exporta um modelo do TF Lite que você pode executar em um dispositivo móvel ou sistema embarcado (como o Raspberry Pi). Além disso, exporta o modelo treinado como um SavedModel do TensorFlow.

Este notebook também aceita um dataset personalizado de arquivos WAV, carregados no Colab em um arquivo ZIP. Quanto mais amostras você tiver para cada classe, maior será a exatidão, mas, como o processo de aprendizado por transferência usa embeddings de características do modelo pré-treinado, você ainda pode conseguir um modelo bastante exato com apenas algumas dezenas de amostras para cada uma das classes.

**Observação:** o modelo que vamos treinar é otimizado para reconhecimento de fala com amostras de um segundo. Se você quiser fazer uma classificação de áudio mais genérica (como detectar diferentes tipos de música), sugerimos que confira [este Colab para retreinar um classificador de áudio](https://colab.sandbox.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb).

Se você quiser executar o notebook com o dataset de falas padrão, pode executar agora mesmo clicando em **Runtime &gt; Run all** (Runtime &gt; Executar tudo) na barra de ferramentas do Colab. Porém, se você quiser usar seu próprio dataset, prossiga para a seção [Prepare o dataset](#scrollTo=cBsSAeYLkc1Z) e siga as instruções.


### Importe os pacotes necessários


Você precisará do TensorFlow, TF Lite Model Maker e alguns módulos para manipulação e reprodução de áudio, além de visualização.

In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

In [ ]:
import os
import glob
import random
import shutil

import librosa
import soundfile as sf
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
from tflite_model_maker.config import ExportFormat

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## Prepare o dataset

Para treinar com o dataset de falas padrão, basta executar todo o código abaixo, sem fazer alterações.

Porém, se você quiser treinar com seu próprio dataset de falas, siga estas etapas:

**Observação:** o modelo que você vai retreinar espera que os dados de entrada sejam áudios de cerca de um segundo com 44,1 kHz. O Model Maker faz a reamostragem automática do dataset de treinamento, então você não precisa fazer a reamostragem do dataset se ele tiver uma taxa de amostragem diferente de 44,1 kHz. Mas saiba que as amostras de áudio com mais de um segundo serão divididas em diversos pedaços de um segundo, e o pedaço final será descartado se tiver duração menor do que um segundo.

1. Confirme se cada amostra do seu dataset está no **formato de arquivo WAV com cerca de um segundo de duração**. Em seguida, crie um arquivo ZIP com todos os arquivos WAV organizados em subpastas separadas para cada classificação. Por exemplo: cada amostra de um comando de voz "yes" (sim) deve estar em uma subpasta chamada "yes". Mesmo se você tiver somente uma classe, as amostras precisam ser salvas em um subdiretório cujo nome seja o nome da classe (esse script pressupõe que seu dataset **não esteja dividido** em conjuntos de treinamento/validação/teste e faz a divisão para você).
2. Clique na guia **Files** (Arquivos) no painel esquerdo e arraste seu arquivo ZIP para lá para carregá-lo.
3. Use a seguinte opção do menu suspenso para definir **`use_custom_dataset`** como True (verdadeiro).
4. Em seguida, vá à seção [Prepare um dataset de áudio personalizado](#scrollTo=EobYerLQkiF1) para especificar seu nome de arquivo ZIP e nome do diretório do dataset.


In [ ]:
use_custom_dataset = False #@param ["False", "True"] {type:"raw"}

### Gere um dataset de ruídos de fundo

Não importa se você esteja usando o dataset de falas padrão ou um personalizado, precisa ter um bom conjunto de ruídos de fundo para que o modelo consiga distinguir falas de outros ruídos (incluindo silêncio).

Como as amostras de fundo abaixo são fornecidas em arquivos WAV com um minuto ou mais, precisamos dividi-las em amostras menores, de um segundo, para reservarmos alguns para o dataset de teste. Também vamos combinar algumas fontes de amostra diferentes para criar um conjunto abrangente de ruídos de fundo e silêncio:

In [ ]:
tf.keras.utils.get_file('speech_commands_v0.01.tar.gz',
                        'http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                        cache_dir='./',
                        cache_subdir='dataset-speech',
                        extract=True)
tf.keras.utils.get_file('background_audio.zip',
                        'https://storage.googleapis.com/download.tensorflow.org/models/tflite/sound_classification/background_audio.zip',
                        cache_dir='./',
                        cache_subdir='dataset-background',
                        extract=True)


**Observação:** embora exista uma versão mais nova disponível, estamos usando a v0.01 do dataset de comandos de voz porque é menor. A v0.01 inclui 30 comandos, enquanto a v0.02 adiciona mais cinco ("backward" – para trás, "forward" – para frente, "follow" – siga, "learn" – aprenda e "visual").

In [ ]:
# Create a list of all the background wav files
files = glob.glob(os.path.join('./dataset-speech/_background_noise_', '*.wav'))
files = files + glob.glob(os.path.join('./dataset-background', '*.wav'))

background_dir = './background'
os.makedirs(background_dir, exist_ok=True)

# Loop through all files and split each into several one-second wav files
for file in files:
  filename = os.path.basename(os.path.normpath(file))
  print('Splitting', filename)
  name = os.path.splitext(filename)[0]
  rate = librosa.get_samplerate(file)
  length = round(librosa.get_duration(filename=file))
  for i in range(length - 1):
    start = i * rate
    stop = (i * rate) + rate
    data, _ = sf.read(file, start=start, stop=stop)
    sf.write(os.path.join(background_dir, name + str(i) + '.wav'), data, rate)

### Prepare o dataset de comandos de voz

Já baixamos o dataset de comandos de voz, então agora só precisamos eliminar um determinado número de classes para nosso modelo.

Esse dataset inclui mais de 30 classificações de comando de voz, e a maioria delas tem mais de 2.000 amostras. Porém, como estamos usando aprendizado por transferência, não precisamos de tantas amostras assim. Dessa forma, o código abaixo faz o seguinte:

- Especifica quais classificações queremos usar e exclui as restantes.
- Mantém somente 150 amostras de cada classe para treinamento (para comprovar que o aprendizado por transferência funciona bem mesmo com datasets menores e reduz o tempo de treinamento).
- Cria um diretório separado para um dataset de teste para que possamos executar a inferência posteriormente com facilidade.

In [ ]:
if not use_custom_dataset:
  commands = [ "up", "down", "left", "right", "go", "stop", "on", "off", "background"]
  dataset_dir = './dataset-speech'
  test_dir = './dataset-test'

  # Move the processed background samples
  shutil.move(background_dir, os.path.join(dataset_dir, 'background'))   

  # Delete all directories that are not in our commands list
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    name = os.path.basename(os.path.normpath(dir))
    if name not in commands:
      shutil.rmtree(dir)

  # Count is per class
  sample_count = 150
  test_data_ratio = 0.2
  test_count = round(sample_count * test_data_ratio)

  # Loop through child directories (each class of wav files)
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[sample_count:sample_count + test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    # Delete remaining samples
    for file in files[sample_count + test_count:]:
      os.remove(file)

### Prepare um dataset personalizado

Se você quiser treinar o modelo com seu próprio dataset de falas, precisa carregar suas amostras como arquivos WAV em um arquivo ZIP ([conforme descrito acima](#scrollTo=cBsSAeYLkc1Z)) e modificar as seguintes variáveis para especificar seu dataset:

In [ ]:
if use_custom_dataset:
  # Specify the ZIP file you uploaded:
  !unzip YOUR-FILENAME.zip
  # Specify the unzipped path to your custom dataset
  # (this path contains all the subfolders with classification names):
  dataset_dir = './YOUR-DIRNAME'

Após alterar o nome do arquivo e o caminho acima, você já pode treinar o modelo com seu dataset personalizado. Na barra de ferramentas do Colab, selecione **Runtime &gt; Run all** (Runtime &gt; Executar tudo) para executar todo o notebook.

O código abaixo integra nossas novas amostras de ruídos de fundo ao dataset e depois separa uma parte de todas as amostras para criar um dataset de teste.

In [ ]:
def move_background_dataset(dataset_dir):
  dest_dir = os.path.join(dataset_dir, 'background')
  if os.path.exists(dest_dir):
    files = glob.glob(os.path.join(background_dir, '*.wav'))
    for file in files:
      shutil.move(file, dest_dir)
  else:
    shutil.move(background_dir, dest_dir)

In [ ]:
if use_custom_dataset:
  # Move background samples into custom dataset
  move_background_dataset(dataset_dir)

  # Now we separate some of the files that we'll use for testing:
  test_dir = './dataset-test'
  test_data_ratio = 0.2
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    test_count = round(len(files) * test_data_ratio)
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[:test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    print('Moved', test_count, 'images from', class_dir)

### Reproduza uma amostra

Para confirmar se o dataset parece correto, vamos reproduzir uma amostra aleatória do dataset de teste:

In [ ]:
def get_random_audio_file(samples_dir):
  files = os.path.abspath(os.path.join(samples_dir, '*/*.wav'))
  files_list = glob.glob(files)
  random_audio_path = random.choice(files_list)
  return random_audio_path

def show_sample(audio_path):
  audio_data, sample_rate = sf.read(audio_path)
  class_name = os.path.basename(os.path.dirname(audio_path))
  print(f'Class: {class_name}')
  print(f'File: {audio_path}')
  print(f'Sample rate: {sample_rate}')
  print(f'Sample length: {len(audio_data)}')

  plt.title(class_name)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

In [ ]:
random_audio = get_random_audio_file(test_dir)
show_sample(random_audio)

## Defina o modelo

Ao usar o Model Maker para retreinar qualquer modelo, você precisa começar pela definição de uma especificação do modelo. A especificação define o modelo base do qual o novo modelo extrairá os embeddings de características para começar a aprender novas classes. A especificação para este reconhecedor de fala é baseado no [modelo BrowserFft pré-treinado do TFJS](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer).

O modelo espera que a entrada seja uma amostra de áudio com 44,1 kHz e com menos de um segundo: o tamanho exato da amostra deve ser de 44.034 quadros.

Você não precisa fazer reamostragem do dataset de treinamento, pois o Model Maker cuida dessa tarefa. Mas, quando você executar a inferência posteriormente, deve garantir que a entrada tenha o formato esperado.

Você só precisa instanciar [`BrowserFftSpec`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/BrowserFftSpec):


In [ ]:
spec = audio_classifier.BrowserFftSpec()

## Carregue seu dataset 

Agora você precisa carregar seu dataset de acordo com as especificações do modelo. O Model Maker inclui a API [`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/DataLoader), que carrega seu dataset a partir de uma pasta e garante que ele esteja no formato esperado segundo a especificação do modelo.

Já reservamos alguns arquivos de teste quando os movemos para um diretório separado, o que facilita a execução da inferência posteriormente. Agora, vamos criar um `DataLoader` para cada divisão: o dataset de treinamento, validação e teste.

#### Carregue o dataset de comandos de voz


In [ ]:
if not use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)

#### Carregue um dataset personalizado

**Observação:** é importante definir `cache=True` para acelerar o treinamento (principalmente quando o dataset for reamostrado), mas isso exige mais RAM para armazenar os dados. Se você estiver utilizando um dataset personalizado muito grande, é possível que o cache exceda a capacidade de RAM.

In [ ]:
if use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)


## Treine o modelo


Agora vamos usar a função [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) do Model Maker para criar um modelo baseado na nossa especificação de modelo e dataset de treinamento. Em seguida, vamos começar o treinamento.

Se você estiver utilizando um dataset personalizado, talvez queira alterar o tamanho do lote para o número de amostras em seu dataset de treinamento, conforme apropriado.

**Observação:** a primeira época demora mais tempo porque é preciso criar o cache. 

In [ ]:
# If your dataset has fewer than 100 samples per class,
# you might want to try a smaller batch size
batch_size = 25
epochs = 25
model = audio_classifier.create(train_data, spec, validation_data, batch_size, epochs)

## Avalie o desempenho do modelo

Mesmo que a exatidão e a perda pareçam boas na saída do treinamento acima, é importante executar o modelo usando os dados de teste que o modelo nunca viu. É isso que o método `evaluate()` faz:

In [ ]:
model.evaluate(test_data)

### Confira a matriz de confusão

Ao treinar um modelo de classificação como este, também vale a pena inspecionar a [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix), que fornece uma representação visual detalhada do desempenho do classificador para cada classificação nos dados de teste.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  sns.set(rc = {'figure.figsize':(6,6)})
  sns.heatmap(
      confusion_normalized, xticklabels=test_labels, yticklabels=test_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True, cbar=False)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## Exporte o modelo

A última etapa é exportar seu modelo para o formato do TensorFlow Lite para execução em dispositivos embarcados/para dispositivos móveis e para o [formato SavedModel](https://www.tensorflow.org/guide/saved_model) para execução em outros lugares.

Ao exportar um arquivo do `.tflite` pelo Model Maker, ele inclui [metadados do modelo](https://www.tensorflow.org/lite/inference_with_metadata/overview), que descrevem diversos detalhes que podem ajudar posteriormente durante a inferência. Ele inclui até mesmo uma cópia do arquivo de rótulos de classificação e, portanto, você não precisa de um arquivo `labels.txt` separado (na próxima seção, mostraremos como usar esses metadados para executar a inferência).

In [ ]:
TFLITE_FILENAME = 'browserfft-speech.tflite'
SAVE_PATH = './models'

In [ ]:
print(f'Exporing the model to {SAVE_PATH}')
model.export(SAVE_PATH, tflite_filename=TFLITE_FILENAME)
model.export(SAVE_PATH, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## Execute a inferência com o modelo do TF Lite

Agora seu modelo do TF Lite pode ser implantado e executado usando qualquer uma das [bibliotecas de inferência](https://www.tensorflow.org/lite/guide/inference) disponíveis ou com a nova [API TFLite AudioClassifier Task](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier). O código abaixo mostra como executar a inferência com um modelo `.tflite` no Python.

In [ ]:
# This library provides the TFLite metadata API
! pip install -q tflite_support

In [ ]:
from tflite_support import metadata
import json

def get_labels(model):
  """Returns a list of labels, extracted from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  labels_file = displayer.get_packed_associated_file_list()[0]
  labels = displayer.get_associated_file_buffer(labels_file).decode()
  return [line for line in labels.split('\n')]

def get_input_sample_rate(model):
  """Returns the model's expected sample rate, from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  metadata_json = json.loads(displayer.get_metadata_json())
  input_tensor_metadata = metadata_json['subgraph_metadata'][0][
          'input_tensor_metadata'][0]
  input_content_props = input_tensor_metadata['content']['content_properties']
  return input_content_props['sample_rate']

Para observar o desempenho do modelo com amostras reais, execute o bloco de código abaixo várias vezes. A cada vez, ele vai buscar uma nova amostra de teste e executar a inferência, e você pode ouvir a amostra de áudio abaixo.

In [ ]:
# Get a WAV file for inference and list of labels from the model
tflite_file = os.path.join(SAVE_PATH, TFLITE_FILENAME)
labels = get_labels(tflite_file)
random_audio = get_random_audio_file(test_dir)

# Ensure the audio sample fits the model input
interpreter = tf.lite.Interpreter(tflite_file)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_size = input_details[0]['shape'][1]
sample_rate = get_input_sample_rate(tflite_file)
audio_data, _ = librosa.load(random_audio, sr=sample_rate)
if len(audio_data) < input_size:
  audio_data.resize(input_size)
audio_data = np.expand_dims(audio_data[:input_size], axis=0)

# Run inference
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], audio_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Display prediction and ground truth
top_index = np.argmax(output_data[0])
label = labels[top_index]
score = output_data[0][top_index]
print('---prediction---')
print(f'Class: {label}\nScore: {score}')
print('----truth----')
show_sample(random_audio)

## Baixe o modelo do TF Lite

Agora você pode implantar o modelo do TF Lite em seu dispositivo móvel ou embarcado. Você não precisa baixar o arquivo de rótulos, pois pode recuperar os rótulos usando os metadados do arquivo `.tflite`, conforme demonstrado no exemplo de inferência anterior.

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(tflite_file)

Confira os aplicativos de exemplo completos que realizam a inferência com modelos de áudio do TF Lite no [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) e no [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios).